<a href="https://colab.research.google.com/github/dipanjanS/mastering-intelligent-agents-langgraph-workshop-dhs2025/blob/main/Module-3-Context-Engineering-for-Agentic-AI-Systems/M3LC4_Build_a_Multi_User_Multi_Session_Adaptive_Agentic_AI_Systems_with_Short_and_Long_Term_Memory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Build a Multi-User - Multi-Session Adaptive Agentic AI Systems with Short and Long-Term Memory with LangGraph and LangMem

**How does a conversational agent store memories?**

For example in the ChatGPT platform you can explicitely call out to ChatGPT when it should store something about you.

![](https://i.imgur.com/7vzt6Hf.png)

You can then go and check out all your past memories in the settings in ChatGPT which basically stores it as a list of facts about you and your preferences.

![](https://i.imgur.com/qf4hF1Y.png)



In this notebook, we design and build an advanced **Agentic AI system** that can handle **multiple users**, each with **multiple sessions**, while maintaining both **short-term** (thread-level) and **long-term** (persistent, user-specific) memory. This system leverages the power of [LangGraph](https://github.com/langchain-ai/langgraph) for agent orchestration and [LangMem](https://github.com/langchain-ai/langmem) for structured memory management.

Here is the agent architecture we will be building.

![](https://i.imgur.com/ifmmwPJ.png)



### Key Features Covered:
- **Short-Term Memory**: Session-scoped context maintained across turns using LangGraph’s checkpointer.
- **Long-Term Memory**: User-specific memory stored using LangMem, enabling agents to recall preferences, history, and facts across sessions.
- **Multi-User Support**: Each user has a unique memory namespace and thread-based session context.
- **Tool-Augmented Agent**: The agent is equipped with tools for web search, memory search, and memory update using Tavily and LangMem utilities.
- **Agent Design**: Implements a ReAct-style graph where the agent reasons, invokes tools, observes outputs, and continues reasoning.
- **Mentoring Use Case**: The system prompt turns the agent into a personalized AI tutor that adapts its behavior based on stored user data.
- **Streaming Execution**: Uses `agent.stream(...)` to interactively respond to user queries while updating memory in real time.
- **Multiple Session Runs**: Demonstrates agent behavior across multiple sessions and users, showcasing memory recall, adaptation, and evolution.

This notebook serves as a complete walkthrough for building **adaptive, memory-aware agents** that personalize over time - useful in education, customer support, healthcare, finance, and more.


## Install Dependencies

In [ ]:
!pip install langchain==0.3.27 langchain-community==0.3.27 langchain-openai==0.3.30 langgraph==0.6.5 langmem==0.0.29 --quiet

## Setup Authentication and LLM Client

Here we authenticate and connect to necessary LLM using OpenAI Authentication

In [ ]:
import os
import getpass

# OpenAI API Key (for chat & embeddings)
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key (https://platform.openai.com/account/api-keys):\n")

# Tavily API Key (for web search)
if not os.environ.get("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Enter your Tavily API key (https://app.tavily.com/home):\n")

In [ ]:
!gdown 1dSyjcjlFoZpYEqv4P9Oi0-kU2gIoolMB

## Create Tools for AI Agent

In [ ]:
from langmem import create_manage_memory_tool, create_search_memory_tool
from langchain.tools import tool
from langchain_community.utilities.tavily_search import TavilySearchAPIWrapper
from langchain_community.retrievers import ArxivRetriever
from langchain_openai import ChatOpenAI

# Tool for web search on general health topics
# Tavily Web Search
tavily_search = TavilySearchAPIWrapper()

@tool
def search_web(query: str) -> list:
    """
    Search the web for general or up-to-date information on healthcare topics.

    Inputs:
    - query (str): The search query string. Should describe the healthcare topic or information you want to find.

    Outputs:
    - list: A list of up to 3 formatted strings, each containing:
        - Title of the search result
        - Content extracted from the page
        - Source URL
    """
    results = tavily_search.raw_results(query=query, max_results=3, search_depth='advanced',
                                        include_answer=False, include_raw_content=True)
    docs = results['results']
    docs = [doc for doc in docs if doc.get("raw_content") is not None]
    docs = ['## Title\n'+doc['title']+'\n\n'+'## Content\n'+doc['raw_content']+'\n\n'+'##Source\n'+doc['url'] for doc in docs]
    return docs


namespace = ("agent_memories", "{user_id}")
MEMORY_MGMT_SYS_PROMPT = """Proactively call this tool when you:

1. Identify a new **USER preference**, especially related to:
   - Learning goals and skill development areas
   - Topics of interest and subject matter focus
   - Preferred teaching or mentoring style
   - Communication tone, level of detail, and pacing
   - Formats, examples, or resources the USER finds helpful
2. Receive an explicit USER request to **remember** something or adjust your behavior.
3. Are in an ongoing mentoring conversation and want to **record important context** that will help personalize future guidance.
4. ONLY UPDATE when an older memory needs replacing else keep ADD NEW MEMORIES based on new user informaiton as per point 1.
"""


create_mem_tool = create_manage_memory_tool(namespace=namespace,
                                            instructions=MEMORY_MGMT_SYS_PROMPT)
search_mem_tool = create_search_memory_tool(namespace)
tools = [search_web, create_mem_tool, search_mem_tool]

llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)
llm_with_tools = llm.bind_tools(tools=tools)

## Build the AI Agent

### Add the System Instructions Prompt

In [ ]:
from langchain_core.messages import SystemMessage

AGENT_INSTRUCTIONS = """
You are an **AI Tutor and Mentor** specializing in **AI, Data Science, Generative AI, and Agentic AI**.
Your role is to explain, teach, and guide the USER in a clear, engaging, and personalized way.
Adapt your explanations to the USER’s background, preferences, and learning goals, making complex concepts easy to understand while providing depth when needed.

## Core Responsibilities
- Always search for existing memories first before answering any questions. Do NOT answer questions before checking existing memories.
- When searching for memories, create a **clear, concise, and optimal search query** that captures the key information needed from past interactions instead of directly using the full user message.
- Add any new information about the user or update existing memories as needed based on details mentioned below.
- Answer USER queries accurately and comprehensively by combining:
  - Retrieved memories
  - Your own internal knowledge
  - Web search results when needed
- Adjust style, tone, and complexity based on USER preferences and prior conversations.
- Actively maintain and leverage long-term USER context by **storing new information** and **retrieving relevant past information** during the conversation.
- Use web search when needed to ensure responses are **factual, relevant, and up-to-date**.
- Maintain a respectful, supportive, and encouraging mentoring tone.

## Available Tools and Usage Rules
You must actively use these tools during the conversation as needed:

1. **Manage Memory Tool** (`create_manage_memory_tool`)
   - Store **new** USER preferences, learning goals, or important information context as soon as they are discovered.
   - Only **update** (append or replace) an existing memory if new user information directly CONTRADICTS or REPLACES older stored information.

2. **Search Memory Tool** (`create_search_memory_tool`)
   - Retrieve relevant USER memories (preferences, progress, past discussions) **before answering** if they could improve personalization or continuity.
   - Always construct a **focused and relevant search query** based on the user’s request or current context.

3. **Web Search Tool** (`search_web`)
   - Use when the query requires:
     - Recent or time-sensitive information
     - Additional detailed information or examples
     - Code samples or up-to-date implementations

## Workflow to follow for each user interaction step-by-step:
1. **Search memories first** with `create_search_memory_tool`:
   - Construct an **optimal query** summarizing the key details you need to recall.
   - Use retrieved results to personalize your upcoming response.
2. **Add new memories immediately** with `create_manage_memory_tool` when new details are learned about the USER:
   - Preferences
   - New information about themselves
   - Likes or dislikes
   - Topics they want to learn about
   - Prefer adding new memories unless they directly contradict existing ones, in which case update the memory instead.
3. **Answer the USER’s query** by combining:
   - Retrieved memories for personalization
   - Your internal knowledge of the domain
   - Web search results when needed (if unsure, researching, or the topic requires current information)
4. When using **web search**, always cite your sources.
5. Maintain conversation continuity across sessions by retrieving and applying stored memories.
6. Be proactive in suggesting related topics, resources, or next steps.
7. Use clear, structured explanations with optional technical depth if the USER requests it.

## Style & Tone
- Personalize responses based on stored preferences and context.
- Be adaptable: explain concepts in multiple ways if the USER needs clarification.
- Present information in logical steps or bullet points.
- Balance theory with practical examples, applications, or exercises.
"""

SYS_PROMPT = SystemMessage(content=AGENT_INSTRUCTIONS)

### Create the Agent Graph with Long and Short Term memory

In [ ]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages

class State(TypedDict):
    messages: Annotated[list, add_messages]

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition
from IPython.display import display, Image, Markdown
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.store.memory import InMemoryStore

# Create the node function that handles reasoning and planning using the LLM
def tool_calling_llm(state: State) -> State:
    # Extract the current conversation history from the state
    current_state = state["messages"]

    # Prepend the system instructions to the current message history
    state_with_instructions = [SYS_PROMPT] + current_state

    # Call the LLM to generate a new message (either a response or a tool call request)
    response = [llm_with_tools.invoke(state_with_instructions)]

    # Return the updated state containing the new message
    return {"messages": response}

# Build the graph
builder = StateGraph(State)

# Add nodes
builder.add_node("agent", tool_calling_llm)
builder.add_node("tools", ToolNode(tools=tools))

# Add edges
builder.add_edge(START, "agent")
# Conditional edge
builder.add_conditional_edges(
    "agent",
    tools_condition, # conditional routing function
    {
        "tools": "tools", # If the latest message (result) from LLM is a tool call request -> tools_condition routes to tools
        "__end__": END # If the latest message (result) from LLM is a not a tool call -> tools_condition routes to END
    }
)
builder.add_edge("tools", "agent") # this is the key feedback loop in the agentic system

# Compile Agent Graph
short_term_memory = InMemorySaver()
long_term_memory_store = InMemoryStore(
    index={
        "dims": 1536,
        "embed": "openai:text-embedding-3-small"
    }
)
agent = builder.compile(checkpointer=short_term_memory,
                        store=long_term_memory_store)

In [ ]:
agent

### Run and Test the Agent

In [ ]:
from agent_utils import format_message

def call_mentor_agent(agent, query, user_id, session_id, verbose=False):

    for event in agent.stream(
        input={"messages": [{"role": "user", "content": query}]},
        config={"configurable": {"user_id": user_id, "thread_id": session_id}},
        stream_mode='values' #returns full agent state with all messages including updates
    ):
        if verbose:
            format_message(event["messages"][-1])

    print('\n\nFinal Response:\n')
    display(Markdown(event["messages"][-1].content))
    return event["messages"]

#### User 1 - Session 1

In [ ]:
user = 'user_dj'
user_session = 'dj-session-001'
query = """Hi, I'm Dipanjan but you can call me DJ,
Please remember I'm proficient in AI, Generative AI and Agentic AI.
I also prefer working in Python and right now

My first question is - Explain CoT prompting in brief
"""
response = call_mentor_agent(agent, query, user, user_session, verbose=True)

In [ ]:
user_session = 'dj-session-001'
query = """Please remember I am working on healthcare insurance projects these days,
so examples from that domain is helpful in the future.

Research on what is a ReAct agent and explain to me with a simple example please
"""
response = call_mentor_agent(agent, query, user, user_session, verbose=True)

In [ ]:
long_term_memory_store.search(("agent_memories",))

#### User 1 - Session 2

In [ ]:
user = 'user_dj'
user_session = 'dj-session-002'
query = """hey, do you remember me?
"""
response = call_mentor_agent(agent, query, user, user_session, verbose=True)

In [ ]:
user_session = 'dj-session-002'
query = """Please remember that I am now working on a financial project in Generative AI so please show me examples only for this domain.

Explain to me what is Agentic AI vs. Generative AI with some examples
"""
response = call_mentor_agent(agent, query, user, user_session, verbose=True)

In [ ]:
long_term_memory_store.search(("agent_memories",))

#### User 1 - Session 3

In [ ]:
user = 'user_dj'
user_session = 'dj-session-003'
query = """Tell me what you know about me.

Also give me a simple example of machine learning vs deep learning
"""
response = call_mentor_agent(agent, query, user, user_session, verbose=True)

In [ ]:
long_term_memory_store.search(("agent_memories",))

#### User 2 - Session 1

In [ ]:
user = 'user_mike'
user_session = 'mm-session-001'
query = """Hi I'm Mike Murdock but you can call me daredevil.
I love fighting crime and working with other superheroes.

Can you tell me which other superheroes are really popular in Marvel?
"""
response = call_mentor_agent(agent, query, user, user_session, verbose=True)

In [ ]:
user_session = 'mm-session-001'
query = """My friends include the fantastic four and spiderman, please remember this.

Do you know any other superheroes I can work with together?
"""
response = call_mentor_agent(agent, query, user, user_session, verbose=True)

In [ ]:
long_term_memory_store.search(("agent_memories",))

In [ ]:
session = 'mm-session-001'
query = """Please add Iron Man and Black Panther to my list of friends also
"""
response = call_mentor_agent(agent, query, user, session, verbose=True)

In [ ]:
long_term_memory_store.search(("agent_memories",))

#### User 2 - Session 2

In [ ]:
user = 'user_mike'
user_session = 'mm-session-002'
query = """Hi! There is a huge threat, Galactus is coming to earth and we need to stop him!

Search my friend list and tell me who could I call for helping me out to protect the world!
"""
response = call_mentor_agent(agent, query, user, user_session, verbose=True)

In [ ]:
long_term_memory_store.search(("agent_memories",))